## Outline:
### 1. Input transcripts and grammars

### 2. Sentencizer transcripts

### 3. Mapping sentences to each category

### 4. Calculate politeness score for each part

In [1]:
# Load packages
import nltk
import spacy

# Load customized packages
from customized_pos import *
from pos_helper import *
from aspect_matching import *
from politeness_helper import *

# Load spacy nlp model
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe(pos_postprocessor_pipe, name="pos_postprocessor", after='tagger')

In [2]:
# Load standardizer
with open('other_collection/standardizer.pkl', 'rb') as f:
    standardizer = pickle.load(f)    
# Load custom model/data
xgb_model = pickle.load(open('model_collection/final_xgb_v2.sav', 'rb'))

### Section 1: Input transcripts and grammars

In [3]:
transcripts = """ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and im actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty"""

In [4]:
grammars = r""" 
    NP: {<PRP.*|DT|JJ|NN.*>+}
    PP: {<IN|TO|RP><NP|VB.*>} 
    VP: {<VB.*|RB.*>+<PP|NP>*}
    Sentence: {<UH>*<NP>+<MD>*<VP|PP|NP>+}
    Sentence: {<PP|VP|NP>+}
    Question: {<MD|WDT|DP|WRB|><MD>*<Sentence|NP|PP|VP>}
"""

### Section 2: Sentencizer transcripts

In [5]:
full_result = nlp_sentencizer(transcripts, grammars, nlp)
text_result = list(map(lambda x: x[1], full_result))
print("full result: ", full_result)
print("\n")
print("only text result: ", text_result)

full result:  [('Sentence', 'ya hello good afternoon speak to nanny seah'), ('Sentence', 'please afternoon miss nanny my name is jaguar shao and'), ('Sentence', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and'), ('Sentence', 'as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty')]


only text result:  ['ya hello good afternoon speak to nanny seah', 'please afternoon miss nanny my name is jaguar shao and', 'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and', 'as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty']


### Section 3: Mapping sentences to each category (standard greetings, state purpose, ask for permission to speak)

In [6]:
category_ls = batch_match_category(text_result, nlp, "greeting")

In [7]:
category_ls

[('opening', 'ya hello good afternoon speak to nanny seah'),
 ('opening', 'please afternoon miss nanny my name is jaguar shao and'),
 ('purpose_of_call',
  'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and'),
 ('purpose_of_call',
  'as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty')]

In [8]:
new_category_ls = nlp_aspect_matching(text_result, nlp, 'greeting', True)
new_category_ls

[('opening',
  'ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and'),
 ('purpose_of_call',
  'ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty')]

### Section 4: Calculate politeness score for each part

In [9]:
politeness_ls = nlp_politeness(new_category_ls, xgb_model, nlp, standardizer, "int", 0.45)
politeness_ls

[('opening',
  'ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and',
  'polite'),
 ('purpose_of_call',
  'ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty',
  'polite')]

### Print result

In [10]:
for category, sentence, politeness in politeness_ls:
    print("Current Category: ", category)
    print("Current Sentence: ", sentence)
    print("Current Politeness Score: ", politeness)
    print("\n")

Current Category:  opening
Current Sentence:  ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and
Current Politeness Score:  polite


Current Category:  purpose_of_call
Current Sentence:  ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty
Current Politeness Score:  polite


